In [ ]:
import numpy as np
import pandas as pd 
import matplotlib as plt 
from pylab import * 
from scipy.optimize import minimize 
from numpy import linalg as LA

## Question 1 
#### Classical simulated annealing. We will use the Schwefel function for D=10 in order to find its global minimum using CSA 𝑓(𝑥1,𝑥2...𝑥𝐷)=418.9829xD−∑ 𝑥𝑖𝐷𝑖 (sin(√𝑥𝑖))  𝑥𝑖 ∈[−500,500] for 𝑖 =1,...,𝐷. In which we use the visitation function of a random displacement along each dimension 𝑥𝑖 =𝑥𝑖 +(2∗𝑈𝑅𝑁−1)×∆, with ∆=0.5 for 𝑖 =1,...,𝐷 

##### a) Fill  in  the  blanks  in  the  provided  simulated  annealing  code.  Use  a  linear  (Tt+1=Tt-α)  cooling schedule with α=0.5, and initializing TSA=3000K, to perform CSA until the temperature reaches 30K and 10K,  and  record  the  function  values.  How  long  is  your  cooling  schedule?  Check  against  the  debugging outputs. Given the stochastic nature of CSA, it would be best to report at least 3 runs for each lower bound temperature. Do you find better solutions when cooling to the lower temperature

In [ ]:
def schwefel(x):
    return 418.9829*len(x)-np.sum(x*np.sin(np.sqrt(np.abs(x))))

In [ ]:
def SA(solution,evaluation,delta,boundary,cooling_schedule):
    """ Simulated Annealing for minimization
    solution: np.array. Initial guess of solution
    evaluation: func. Function to evaluate solution
    delta: float. Magnitude of random displacement
    boundary: array of int/float. [lowerbound,upperbound]
    cooling_schedule: np.array. An array of tempretures for simulated annealing
    """
    best_solution=solution.copy()
    lowest_eval=evaluation(best_solution)
    for idx,temp in enumerate(cooling_schedule):
        if idx%500==0:
            print("%d/%d   temp:%f"%(idx,len(cooling_schedule),temp))
        for n in range(len(solution)):
            trial=solution.copy()
            trial[n]+=delta*(2*np.random.random()-1)
            if trial[n]>=boundary[0] and trial[n]<=boundary[1]:
                #fill in acceptance criterion
                if np.exp(-(evaluation(trial) - evaluation(solution))/temp) > np.random.random():
                    solution=trial
                    if evaluation(solution)<lowest_eval:
                        #update solution here
                        best_solution = solution.copy()
                        lowest_eval = evaluation(solution)
    return {"solution":best_solution,"evaluation":lowest_eval}

In [ ]:
starting_point = np.random.random(10)
#linear cooling schedule with alpha = 0.5 and initial temperature 3000K 
cooling_schedule=np.arange(3000,10,-0.5)
print(f"The cooling schedule is {len(cooling_schedule)}K for 10k")
SA(starting_point, schwefel, 0.5, [-500,500], cooling_schedule)

In [ ]:
cooling_schedule=np.arange(3000,30,-0.5)
print(f"The cooling schedule is {len(cooling_schedule)}K for 10k")
SA(starting_point, schwefel, 0.5, [-500,500], cooling_schedule)

##### The cooling schedule was 5980K for 10K and 5940K for 30K. After running the both the temperatures three times, the following results were retrieved. 

##### 10K: [4175, 4150,4166] = average : 4163
##### 30K: [4129, 4052,4156] = average : 4112 

##### Both values are very comparable but from average, it seems that the lower temperature yields better evaluation. With additional studies, this might change. 

##### b) Choose logarithmic cooling (Tk=TSA/(1+ TSA log(1+k)/3curr), where k is counter for number of cooling cycle) and curr is an adjustable parameter,  with two initial temperature TSA = 3000K and 6000K. Use curr = 1000 and k = 6000. Reconsider questions (a). Do these cooling schedules converge better than linear cooling? 

In [ ]:
def log_cooling_schedule(T, sigma, k):
    cooling_schedule = []
    for i in range(k):
        cooling_schedule.append(T)
        T = T/(1+(T*np.log(1+i)/(3*sigma)))
    return cooling_schedule


In [ ]:
SA(starting_point, schwefel, 0.5, [-500,500], log_cooling_schedule(3000, 1000, 6000))

In [ ]:
SA(starting_point, schwefel, 0.5, [-500,500], log_cooling_schedule(6000, 1000, 6000))

##### The following averages were calcualted for both the starting temperatures and their evaluations 

##### 3000K = [4110, 4130, 4110] = 4116 
##### 6000K = [4110, 4130, 4089] = 4109 

##### The results for logarithmic calculations seemed to be doing better overall when compared to linear cooling.

##### c) Create  your  own  annealing  schedule  (cooling  and  heating  cycles)  to  see  if  you  can  find  better solutions. Use a local optimization technique on your CSA answer, can you find even better solution? 

In [ ]:
schedule=np.append(cooling_schedule,np.linspace(cooling_schedule[-1],100,200))
schedule=np.append(schedule,np.linspace(schedule[-1],1000,2000))
schedule=np.append(schedule,np.linspace(schedule[-1],500,100))
schedule=np.append(schedule,np.linspace(schedule[-1],5000,3000))
schedule=np.append(schedule,np.linspace(schedule[-1],400,100))
schedule=np.append(schedule,np.linspace(schedule[-1],4000,3000))
schedule=np.append(schedule,np.linspace(schedule[-1],900,100))
schedule=np.append(schedule,np.linspace(schedule[-1],9000,2000))
plt.figure()
plt.plot(schedule)

In [ ]:
#CG methods 
res = minimize(schwefel, starting_point , method='CG', options={'disp':True, 'gtol':1e-5})

##### The solution is not better than the original solution. Additional analysis of the scenario may lead to better results. 

## Question 2 

### Clustering is a widely used technique in exploratory data analysis that we will examine later using unsupervised learning for classification of objects into groups. But for now we will consider a popular meta-heuristic for solving it using CSA. In this case we would like to cluster N data points into K clusters by solving the minimization of the following cost function:  𝐽(𝑁,𝐾)=∑∑𝑤𝑖𝑗𝑑𝑖𝑗2𝐾𝑗=1𝑁𝑖=1    𝑤𝑖𝑗 ={1  𝑖𝑓 𝑝𝑜𝑖𝑛𝑡 𝑖 𝑖𝑠 𝑎𝑠𝑠𝑖𝑔𝑛𝑒𝑑 𝑡𝑜 𝑐𝑙𝑢𝑠𝑡𝑒𝑟 𝑗0 𝑜𝑡ℎ𝑒𝑟𝑤𝑖𝑠𝑒, 1≤𝑖 ≤𝑁    𝑎𝑛𝑑    1≤𝑗 ≤𝐾  where 𝑑𝑖𝑗 is the Euclidean distance between point 𝑖 and the center of cluster 𝑗, and condition on 𝑤𝑖𝑗 ensures that a point is defined to be in one of the distinct clusters 𝐾.  

In [ ]:
df = pd.read_csv('wines.csv')
rank = df['ranking'].tolist()
df.drop('ranking', axis=1, inplace=True)
df.tail()

##### a) Normalize your chemical descriptor data for each attribute by subtracting off the mean and dividing by the standard deviation.

In [ ]:
# calculate the mean of each column 
mean = df.mean(axis=0)
# calculate the standard deviation of each column
std = df.std(axis=0)
# subtract the mean and divide by the standard deviation
df_mean = (df - mean) / std
df_mean

##### b) Given the initial categorization of the 178 wines into the 3 clusters according to Start assignment column in the dataset, determine the centroid of each of the three clusters. The centroid for this problem is a 13-D vector where each entry is the mean of a variable for the observations in that cluster.

In [ ]:
center = df_mean.mean()

##### c) Given the centroid, determine the value of the cost function for this initial categorization. Check against the debugging output. 

In [ ]:
def cost(centers, feats, ranks):
    """ Cost function for clustering
    centers: np.array shape (3,13). Fixed centers
    feats: pd.DataFrame. Normalized chemical descriptors
    ranks: np.array shape(178,). Assignment.
    """
    cost = 0
    for i in range(len(ranks)):
        cost += LA.norm(feats.iloc[i] - centers[int(ranks[i])]) ** 2
    return cost

cost(center, df_mean, rank)

##### d) Fill in the blanks in the provided simulated annealing code. Use CSA with a visitation function in which a randomly chosen wine 𝑖 is moved from its present cluster 𝑗 to another randomly chosen cluster 𝑘 ≠𝑗. One epoch corresponds to attempting to move all 𝑁 wines between clusters, i.e. there are 𝑁 Metropolis steps, at each temperature. Use a start temperature of 500, and use a geometric cooling schedule(Tt+1=αTt)  with  α=0.999  and  total  of  5000  steps,  again  using  at  least  3  runs  of  CSA.  Check  your  final  temperature against  debugging  output.  Report  all  3  solutions  and  the  wine  members  as  part  of  each  cluster.  Validate your result using the provided code. How well is the assignment?

##### I was not able to complete this question as my computer is really slow and was not able to analyze this data. I tried running it for a long amount of time but it did not yield any result beyond the first output. 

In [ ]:
def simulated_annealing(feats,ranks,centers,start_temp,alpha,steps=10000):
    """ Simulated Annealing for clustering
    feats: pd.DataFrame. Normalized chemical descriptors
    ranks: np.array shape(178,). Initial assignment.
    centers: np.array shape (3,13). Fixed centers
    start_temp: float. Initial tempreture
    alpha: float. Hyperparameter for geometric cooling
    steps: int. 
    """
    best_rank=ranks.copy()
    # evaluate the cost function with current best rank
    lowest_eval=cost(centers,feats,best_rank)
    for step in (range(steps)):
        # update tempture according to geometric cooling schedule
        temp=start_temp*alpha**step
        if step%500==0:
            print(step,temp,lowest_eval)
        for n in range(len(ranks)):
            trial=ranks.copy()
            rand_choice=np.random.randint(3)+1
            trial[n]=rand_choice
            # Metropolis acceptance criterion
            if np.exp(-(cost(centers, feats, trial) - cost(centers, feats, ranks)) / start_temp) > np.random.random():
                ranks=trial
                # update evaluation
                new_eval=cost(centers,feats,ranks)
                if new_eval<lowest_eval:
                    #update best rank and lowest_eval
                    best_rank=ranks.copy()
                    lowest_eval=new_eval
    return {"solution":best_rank,"evaluation":lowest_eval}

##### e) Adapt  your  code  in  2(d).  Now  use  CSA  with  a  visitation  function  in  which  a  randomly  chosen centroid 𝑗 is updated as a random walk for each of its 13 components  𝑥𝑖 =𝑥𝑖 +(2∗𝑈𝑅𝑁−1)×∆; with ∆=0.01 In this case one epoch corresponds to moving all 𝐾 =3 cluster centers at each temperature, reassigning all wines to their nearest centroid, and evaluating the new cost function. Check against the debugging output to make sure you are assigning wines correctly. Use a start temperature of 500, and use a geometric cooling schedule with α=0.999 and total of 5000 steps, again using at least 3 runs of CSA. Report all 3 solutions and the wine members as part of each cluster. Is this a better solution than found in (d)?  
 

##### I was not able to complete this question as my computer is really slow and was not able to analyze this data. I tried running it for a long amount of time but it did not yield any result beyond the first output. See above. 

In [ ]:
def simulated_annealing_b(feats,ranks,centers,start_temp,alpha,steps=10000):
    """ Simulated Annealing for clustering
    feats: pd.DataFrame. Normalized chemical descriptors
    ranks: np.array shape(178,). Initial assignment.
    centers: np.array shape (3,13). Fixed centers
    start_temp: float. Initial tempreture
    alpha: float. Hyperparameter for geometric cooling
    steps: int. 
    """
    best_rank=ranks.copy()
    # evaluate the cost function with current best rank
    lowest_eval=cost(centers,feats,best_rank)
    for step in (range(steps)):
        # update tempture according to geometric cooling schedule
        temp=start_temp*alpha**step
        if step%500==0:
            print(step,temp,lowest_eval)
        for n in range(len(ranks)):
            trial=ranks.copy()
            rand_choice=np.random.randint(3)+1
            trial[n]+= (2 * rand_choice - 1) * delta
            # Metropolis acceptance criterion
            if np.exp(-(cost(centers, feats, trial) - cost(centers, feats, ranks)) / start_temp) > np.random.random():
                ranks=trial
                # update evaluation
                new_eval=cost(centers,feats,ranks)
                if new_eval<lowest_eval:
                    #update best rank and lowest_eval
                    best_rank=ranks.copy()
                    lowest_eval=new_eval
    return {"solution":best_rank,"evaluation":lowest_eval}